In [1]:
seed = 1

import os
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

import random as rn
rn.seed(seed)

import tensorflow as tf
tf.random.set_seed(seed)


In [2]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np
np.random.seed(seed)

In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print("train_images shape:", x_train.shape)
print("test_images shape:", x_test.shape)
print("train_labels shape:", y_train.shape)
print("test_labels shape:", y_test.shape)

# train_images = train_images / 255
# test_images = test_images / 255

train_images shape: (50000, 32, 32, 3)
test_images shape: (10000, 32, 32, 3)
train_labels shape: (50000, 1)
test_labels shape: (10000, 1)


In [9]:
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)
 
num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [11]:
def lr_schedule(epoch):
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    if epoch > 100:
        lrate = 0.0003
    return lrate
    
weight_decay = 1e-4
batch_size = 64

In [12]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 32)        896       
_________________________________________________________________
activation (Activation)      (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 32)       

In [13]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

In [19]:
num_epochs = 150
opt_rms = keras.optimizers.RMSprop(learning_rate=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] // batch_size, epochs = num_epochs,
                    verbose=1, validation_data=(x_test,y_test), callbacks=[LearningRateScheduler(lr_schedule)])
#save to disk
# model_json = model.to_json()
# with open('model.json', 'w') as json_file:
#     json_file.write(model_json)
# model.save_weights('model.h5') 
 
#testing



/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/150
781/781 [==============================] - 29s 34ms/step - loss: 0.7497 - accuracy: 0.7802 - val_loss: 0.6834 - val_accuracy: 0.8127
Epoch 2/150
781/781 [==============================] - 26s 33ms/step - loss: 0.7291 - accuracy: 0.7875 - val_loss: 0.7254 - val_accuracy: 0.8038
Epoch 3/150
781/781 [==============================] - 26s 33ms/step - loss: 0.7124 - accuracy: 0.7979 - val_loss: 0.6641 - val_accuracy: 0.8160
Epoch 4/150
781/781 [==============================] - 26s 33ms/step - loss: 0.6991 - accuracy: 0.8014 - val_loss: 0.6791 - val_accuracy: 0.8217
Epoch 5/150
781/781 [==============================] - 26s 33ms/step - loss: 0.6938 - accuracy: 0.8053 - val_loss: 0.6404 - val_accuracy: 0.8271
Epoch 6/150
781/781 [==============================] - 26s 33ms/step - loss: 0.6912 - accuracy: 0.8055 - val_loss: 0.6295 - val_accuracy: 0.8333
Epoch 7/150
781/781 [==============================] - 26s 33ms/step - loss: 0.6823 - accuracy: 0.8083 - val_loss: 0.6747 - val_ac

In [20]:
scores = model.evaluate(x_train, y_train, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

scores = model.evaluate(x_test, y_test, batch_size=128, verbose=1)
print('\nTest result: %.3f loss: %.3f' % (scores[1]*100,scores[0]))

391/391 [==============================] - 3s 7ms/step - loss: 0.2573 - accuracy: 0.9450

Test result: 94.498 loss: 0.257
79/79 [==============================] - 1s 6ms/step - loss: 0.4308 - accuracy: 0.8914

Test result: 89.140 loss: 0.431
